# Indexing / querying JSON documents
## Adding a JSON document to an index

In [1]:
import redis
from redis.commands.json.path import Path
import redis.commands.search.aggregation as aggregations
import redis.commands.search.reducers as reducers
from redis.commands.search.field import TextField, NumericField, TagField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import NumericFilter, Query


r = redis.Redis(host='localhost', port=36379)
user1 = {
    "user":{
        "name": "Paul John",
        "email": "paul.john@example.com",
        "age": 42,
        "city": "London"
    }
}
user2 = {
    "user":{
        "name": "Eden Zamir",
        "email": "eden.zamir@example.com",
        "age": 29,
        "city": "Tel Aviv"
    }
}
user3 = {
    "user":{
        "name": "Paul Zamir",
        "email": "paul.zamir@example.com",
        "age": 35,
        "city": "Tel Aviv"
    }
}
r.json().set("user:1", Path.root_path(), user1)
r.json().set("user:2", Path.root_path(), user2)
r.json().set("user:3", Path.root_path(), user3)

schema = (TextField("$.user.name", as_name="name"),TagField("$.user.city", as_name="city"), NumericField("$.user.age", as_name="age"))
r.ft().create_index(schema, definition=IndexDefinition(prefix=["user:"], index_type=IndexType.JSON))

b'OK'

## Searching

### Simple search

In [2]:
r.ft().search("Paul")

Result{2 total, docs: [Document {'id': 'user:1', 'payload': None, 'json': '{"user":{"name":"Paul John","email":"paul.john@example.com","age":42,"city":"London"}}'}, Document {'id': 'user:3', 'payload': None, 'json': '{"user":{"name":"Paul Zamir","email":"paul.zamir@example.com","age":35,"city":"Tel Aviv"}}'}]}

### Filtering search results

In [3]:
q1 = Query("Paul").add_filter(NumericFilter("age", 30, 40))
r.ft().search(q1)

Result{1 total, docs: [Document {'id': 'user:3', 'payload': None, 'json': '{"user":{"name":"Paul Zamir","email":"paul.zamir@example.com","age":35,"city":"Tel Aviv"}}'}]}

### Projecting using JSON Path expressions 

In [4]:
r.ft().search(Query("Paul").return_field("$.user.city", as_field="city")).docs

[Document {'id': 'user:1', 'payload': None, 'city': 'London'},
 Document {'id': 'user:3', 'payload': None, 'city': 'Tel Aviv'}]

## Aggregation

In [5]:
req = aggregations.AggregateRequest("Paul").sort_by("@age")
r.ft().aggregate(req).rows

[[b'age', b'35'], [b'age', b'42']]